In [1]:
import re
import pandas as pd
import numpy as np
import os
from RecipeExtractor import EXTRACT_PATTERNS
import RecipeExtractor
from fastavro import writer, reader, parse_schema
from os.path import exists

In [2]:
EXTRACT_PATTERNS

{'bbc food': {'title': [{'findallFirst': '<h1 class="gel-trafalgar content-title__text".*?>(.+?)</h1>'}],
  'servings': [{'findallFirst': '<p class="recipe-metadata__serving".*?>(.+?)</p>'},
   {'sub': ('[^0-9]', '')}]},
 'british': {'title': [{'findallFirst': '<h1 itemprop="name">(.+?)</h1>'}],
  'servings': [{'findallFirst': '<span class="header-attribute-text">(.+?)</span>'},
   {'sub': ('[^0-9]', '')}]},
 'italian': {'title': [{'findallFirst': '<h1 itemprop="name">(.+?)</h1>'}],
  'servings': [{'findallFirst': '<span class="header-attribute-text">(.+?)</span>'},
   {'sub': ('[^0-9]', '')}]},
 'delish': {'title': [{'findallFirst': '<h1 class="content-hed recipe-hed">(.+?)</h1>'}],
  'servings': [{'findallFirst': '<span class="yields-amount">(.+?)</span>'},
   {'sub': ('[^0-9]', '')}]},
 'good_food': {'title': [{'findallFirst': '<h1 class="heading-1">(.+?)</h1>'}],
  'servings': [{'findall3': '<div class="icon-with-text__children">(.+?)</div>'},
   {'sub': ('[^0-9]', '')}]},
 'jamie'

In [4]:
file_dir = "cleaned_csv/"
for file in os.listdir(file_dir):
    if file.endswith(".csv"):
            print("############", file)
            df = pd.read_csv(file_dir+file,sep="\t",index_col=0)
            for index,row in df.iterrows():
                if index == 5:
                    break
                print(row['path_to_file'],row['source'])
                source = row['source'].upper()
                html_path = row['path_to_file']
                with open(html_path, "r", encoding='utf-8') as f:
                    site_content = f.read()
                    site_content = re.sub("[^\S ]+","",site_content)

                    site_patterns = EXTRACT_PATTERNS[source.lower()]

                    title_patterns = site_patterns['title']
                    servings_patterns = site_patterns['servings']

                    title = RecipeExtractor.getTitleFromSiteContent(title_patterns, site_content)
                    print(title)
                    servings = RecipeExtractor.getServingsFromSiteContent(servings_patterns, site_content)
                    print(servings)
                    ingredients = RecipeExtractor.getIngredientsFromSiteContent(site_content,source)
                    print(ingredients)
                    steps = RecipeExtractor.getStepsFromSiteContent(site_content,source)
                    print(steps)

############ bbc_food_clean.csv
./DATA/bbc/Quick Christmas dinner recipe - BBC Food.html BBC FOOD
2-hour Christmas dinner
68
['2.5kg/5lb 8oz turkey crown (fully thawed if frozen)', '6 rashers smoked streaky bacon', 'sunflower oil, for greasing', 'large knob of butter', '2 medium leeks, trimmed', '5 thick slices white bread', '1 x 120g pack mixed nuts and dried fruit (I use one containing brazil nuts, pecans, almonds, sultanas and dried cranberries)', '1 small bunch fresh thyme, finely chopped', '1 large unwaxed lemon, finely grated zest only', '450g/1lb pork sausagemeat', 'salt and freshly ground black pepper', '1.2kg/2lb 12oz potatoes, preferably Maris Piper or King Edward, peeled and cut into large chunks', '4 tbsp sunflower oil or melted goose fat', '8–12 chipolata sausages, wrapped in bacon', 'a few sprigs fresh rosemary (optional)', '2 tbsp runny honey', '750–900ml/1⅓–1⅔ pint readymade chicken gravy', '2 rashers smoked streaky or back bacon, roughly chopped', '4 tbsp ruby port or 

############ good_food_clean.csv
./DATA/good_food/Celebration cake recipes.html good_food
Celebration cake recipes

[]
[]
./DATA/good_food/Pasta recipes.html good_food
Pasta recipes

[]
[]
./DATA/good_food/Chocolate drip cake recipe.html good_food
Chocolate drip cake
2025
['200g unsalted butter, softened, plus extra for greasing', '250g golden caster sugar', '3 large eggs', '225g plain flour', '25g powdered malt drink', '2 tsp baking powder', '100g natural yogurt', '2 tsp vanilla extract', '175g unsalted butter, softened, plus extra for greasing', '250g golden caster sugar', '3 large eggs', '225g plain flour', '25g cocoa powder', '2 tsp baking powder', '50g natural yogurt', '100g dark chocolate, melted and cooled a little', 'butter or flavourless oil, for greasing', '200g golden caster sugar', '5 tbsp golden syrup', '2 tsp bicarbonate of soda', '37g pack Maltesers', '250g pack slightly salted butter , softened', '750g icing sugar', '3 tbsp powdered malt drink', '1 tsp vanilla extract',

[]
[]
./DATA/seriouseats/Easy Roast Chicken with Asparagus and Leeks Recipe.html seriouseats
Easy Roast Chicken with Asparagus and Leeks Recipe
4
['1 whole butterflied chicken (about 4 pounds, see note), neck and backbone reserved', 'Kosher salt and freshly ground black pepper', '4 large leeks, whites and greens reserved separately, whites trimmed, washed, and split in half lengthwise', '2 bay leaves', '1 1/2 pounds asparagus stalks, trimmed and peeled if necessary', '2 tablespoons olive oil']
['Adjust oven rack to center position and preheat oven to 500°F. Pat chicken dry with paper towels and season on all sides generously with salt and pepper. Line heavy-duty rimmed backing sheet with foil. Place chicken on foil, skin-side up. Transfer to oven and roast until thickest part of breast registers 140 to 150°F on an instant-read thermometer, about 45 minutes. Remove from oven, transfer chicken to a cutting board, and allow to rest for 15 minutes.', 'While chicken roasts, make jus. Chop n

In [8]:
schema = {
    'doc': 'Test 10 recipe for site.',
    'name': '10recipe',
    'namespace': 'test',
    'type': 'record',
    'fields': [
        {'name': 'url', 'type': 'string'},
        {'name': 'website', 'type': 'string'},
        {'name': 'content', 'type': 'string'},
    ],
}
parsed_schema = parse_schema(schema)
records = []
file_dir = "cleaned_csv/"
for file in os.listdir(file_dir):
    if file.endswith(".csv"):
            print("############", file)
            df = pd.read_csv(file_dir+file,sep="\t",index_col=0)
            for index,row in df.iterrows():
                if index == 10:
                    break
                source = row['source'].upper()
                html_path = row['path_to_file']
                url = row['url']
                new_record = {}
                if exists(html_path):
                    with open(html_path, "r", encoding='utf-8') as f:
                        site_content = f.read()
                        site_content = re.sub("[^\S ]+","",site_content)
                        new_record['url'] = url
                        new_record['website'] = source
                        new_record['content'] = site_content
                        records.append(new_record)
with open('10recipe.avro', 'wb') as out:
    writer(out, parsed_schema, records)

############ bbc_food_clean.csv
############ british_food_clean.csv
############ delish_food_clean.csv
############ good_food_clean.csv
############ italian_food_clean.csv
############ jamie_clean.csv
############ seriouseats_clean.csv
############ simply_clean.csv
############ spoon_clean.csv


In [6]:
schema = {
    'doc': 'ALl recipes for site.',
    'name': 'Allrecipe',
    'namespace': 'test',
    'type': 'record',
    'fields': [
        {'name': 'url', 'type': 'string'},
        {'name': 'website', 'type': 'string'},
        {'name': 'content', 'type': 'string'},
    ],
}
parsed_schema = parse_schema(schema)

file_dir = "cleaned_csv/"
for file in os.listdir(file_dir):
    if file.endswith(".csv"):
        print("############", file)
        records = []
        df = pd.read_csv(file_dir+file,sep="\t",index_col=0)
        for index,row in df.iterrows():
            source = row['source'].upper()
            html_path = row['path_to_file']
            url = row['url']
            if index % 1000 == 0:
                if exists('allrecipe.avro'):
                    with open('allrecipe.avro', 'a+b') as out:
                        writer(out, parsed_schema, records)
                    records = []
                else:
                    with open('allrecipe.avro', 'wb') as out:
                        writer(out, parsed_schema, records)
                    records = []
            new_record = {}
            if exists(html_path):
                with open(html_path, "r", encoding='utf-8') as f:
                    site_content = f.read()
                    site_content = re.sub("[^\S ]+","",site_content)
                    new_record['url'] = url
                    new_record['website'] = source
                    new_record['content'] = site_content
                    records.append(new_record)
        if exists('allrecipe.avro'):
            with open('allrecipe.avro', 'a+b') as out:
                writer(out, parsed_schema, records)
        else:
            with open('allrecipe.avro', 'wb') as out:
                writer(out, parsed_schema, records)

############ bbc_food_clean.csv
############ british_food_clean.csv
############ delish_food_clean.csv
############ good_food_clean.csv
############ italian_food_clean.csv
############ jamie_clean.csv
############ seriouseats_clean.csv
############ simply_clean.csv
############ spoon_clean.csv


In [9]:
count = 0
with open('10recipe.avro', 'rb') as fo:
    for record in reader(fo):
        count += 1
count

90